In [ ]:
#/////////////////////////////////////////////#
# plot 3D-vendering flow field with particles #
# created by Sisi Chen in June 2019           #
# questions/comments, contact                 #
# sisichen@ucar.edu                           #
#/////////////////////////////////////////////#

#import packages for plotting
from mayavi import mlab
mlab.init_notebook('png')#, 800, 600)
#database & file io
import numpy as np
from netCDF4 import Dataset
import pandas as pd
#import ffmpeg


In [6]:
for i in nframes:
    #initiate figure
    mlab.clf(figure=None)
    mlab.figure(3,fgcolor=(.0,.0,.0), bgcolor=(1.0,1.0,1.0),size=(1920, 1080))
    #flow field
    fig1=mlab.contour3d(yv,zv,xv,fields[i][:][:][:],colormap='rainbow',vmax=fmax,vmin=fmin,contours=15,opacity=0.1)

NameError: name 'mlab' is not defined

In [2]:
#import dynamic field data
dynancfile='/glade/u/home/sisichen/Github/aerosol/git_aerosol/out.ncf'
fh = Dataset(dynancfile,mode='r')
ZX=fh.variables['ZX'][:] #vorticity in x-dir
ZY=fh.variables['ZY'][:]
ZZ=fh.variables['ZZ'][:]
TH=fh.variables['TH'][:] #temperature
QV=fh.variables['QV'][:] #water vapor mixing ratio
fh.close()

In [3]:
#set up mesh grid & domain
l_box=16.5 #domain size 8.3cm
N_grid=128
x1=np.linspace(0,l_box,N_grid)
y1=np.linspace(0,l_box,N_grid)
z1=np.linspace(0,l_box,N_grid)
yv,zv,xv = np.meshgrid(y1,z1,x1,indexing='ij')
#calculate 3D enstrophy field
vort=ZX**2+ZY**2+ZZ**2

In [4]:
#import droplet data using pandas
df = pd.read_csv('droplet_new.track',sep='\s+',header=None,names=['time','id','x','y','z','r','s'])
#save it to csv file
#df.to_csv(r'droplet_track.csv')

In [90]:
#select data
%matplotlib inline
ndrop_total=6400 # total droplet number
index_range = range(0,6401) #select droplets
nframes     = range(0,100) #select time frames
fields=vort #select field datafields=TH
fmax=3e4#fields.max()/10#mean()+(fields.max()-fields.min())/20
fmin=1e3#fields.min()#mean()-(fields.max()-fields.min())/20

In [91]:
#first set up the figure, axis, & plot element
## mlab colormap options
#accent       flag          hot      pubu     set2
#autumn       gist_earth    hsv      pubugn   set3
#black-white  gist_gray     jet      puor     spectral
#blue-red     gist_heat     oranges  purd     spring
#blues        gist_ncar     orrd     purples  summer
#bone         gist_rainbow  paired   rdbu     winter
#brbg         gist_stern    pastel1  rdgy     ylgnbu
#bugn         gist_yarg     pastel2  rdpu     ylgn
#bupu         gnbu          pink     rdylbu   ylorbr
#cool         gray          piyg     rdylgn   ylorrd
#copper       greens        prgn     reds     rainbow
#dark2        greys         prism    set1     gnuplot
for i in nframes:
    #get droplet 3D location
    xdrop=df[df['time']==df.time[i*ndrop_total]].x*1e2 #convert from m to cm by *1e2
    ydrop=df[df['time']==df.time[i*ndrop_total]].y*1e2
    zdrop=df[df['time']==df.time[i*ndrop_total]].z*1e2
    #initiate figure
    mlab.clf(figure=None)
    mlab.figure(3,fgcolor=(.0,.0,.0), bgcolor=(1.0,1.0,1.0),size=(1920, 1080))
    #droplets
    fig1=mlab.points3d(xdrop,ydrop,zdrop,scale_factor=.05,color=(.5,.5,1))
    #flow field
    fig1=mlab.contour3d(yv,zv,xv,fields[i][:][:][:],colormap='rainbow',vmax=fmax,vmin=fmin,contours=15,opacity=0.1)
    mlab.axes(fig1,z_axis_visibility=False,y_axis_visibility=False,x_axis_visibility=False,xlabel=' ', ylabel=' ',zlabel=' ')
    mlab.axes.font_factor=0.1
    mlab.outline() #box on
    mlab.scalarbar(title='Enstrophy',nb_colors=20)
    #mlab.view(azimuth=-20)
    #mlab.move(-15)
    fig1
    #filenames & savefig
    num=str(i).zfill(3)
    figfile="plot"+num+".png"
    mlab.savefig(figfile)
    mlab.clf(figure=None)
    

In [92]:
!ffmpeg -r 10 -f image2  -s 1920x1080 -i plot%03d.png -crf 25 -pix_fmt yuv420p move1.mp4

ffmpeg version 4.1.3 Copyright (c) 2000-2019 the FFmpeg developers
  built with clang version 4.0.1 (tags/RELEASE_401/final)
  configuration: --prefix=/Users/sisichen/anaconda3 --cc=x86_64-apple-darwin13.4.0-clang --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 22.100 / 56. 22.100
  libavcodec     58. 35.100 / 58. 35.100
  libavformat    58. 20.100 / 58. 20.100
  libavdevice    58.  5.100 / 58.  5.100
  libavfilter     7. 40.101 /  7. 40.101
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  3.100 /  5.  3.100
  libswresample   3.  3.100 /  3.  3.100
  libpostproc    55.  3.100 / 55.  3.100
Input #0, image2, from 'plot%03d.png':
  Duration: 00:00:10.00, start: 0.000000, bitrate: N/A
    Stream #0:0: Video: png, rgb24(pc), 40